Dưới đây là một hướng dẫn chi tiết kèm code mẫu cho cách triển khai quy trình tự động tạo prompt, tạo test cases, đánh giá kết quả, nhận feedback từ người dùng và cập nhật prompt. Chúng ta sẽ triển khai:

1. **Backend (FastAPI)**  
   - Tích hợp “4o-mini” để tạo prompt và test cases.  
   - Sử dụng “o1-min” (hoặc một model LLM mạnh hơn) để đánh giá prompt.  
   - Thực hiện vòng lặp tự động (iterative loop) cho đến khi đạt yêu cầu (ví dụ: accuracy ≥ 90% hoặc đạt số vòng tối đa).  
   - Cung cấp endpoint để nhận feedback từ người dùng nhằm cập nhật prompt.

2. **Frontend (React)**  
   - Giao diện cho người dùng nhập dữ liệu (format, samples, conditions).  
   - Hiển thị prompt được tạo, bảng test cases và thông số đánh giá (accuracy, response time, iteration, history).  
   - Cho phép người dùng nhập feedback và gửi về backend.  
   - Cho phép người dùng “update prompt” dựa trên feedback và lặp lại quá trình.

Dưới đây là ví dụ code cho từng phần.

---

## 1. Backend (FastAPI)

### Cấu trúc thư mục mẫu:
```
backend/
├── main.py
├── utils.py         # (các hàm giả lập: generate_prompt, generate_test_cases, evaluate_prompt, update_prompt)
├── models.py        # (định nghĩa Pydantic models)
├── requirements.txt
└── Dockerfile
```

### File: `backend/models.py`
```python
from pydantic import BaseModel
from typing import List, Optional

class Sample(BaseModel):
    input: str
    output: str

class TestCase(BaseModel):
    input: str
    expected_output: str
    actual_output: str
    is_correct: bool
    similarity_score: float

class OptimizationHistory(BaseModel):
    iteration: int
    accuracy: float
    response_time: float

class PromptRequest(BaseModel):
    format: str
    samples: List[Sample]
    conditions: Optional[str] = None
    iteration: int = 0  # vòng lặp hiện tại

class PromptResponse(BaseModel):
    generated_prompt: str
    test_cases: List[TestCase]
    accuracy: float
    response_time: float
    iteration: int
    optimization_history: List[OptimizationHistory]

class FeedbackRequest(BaseModel):
    prompt: str
    feedback: str
```

### File: `backend/utils.py`
Trong file này, chúng ta mô phỏng các hàm sử dụng “4o-mini” để tạo prompt & test cases, và “o1-min” để đánh giá prompt.  
Bạn có thể thay thế các hàm này bằng tích hợp model thực tế.
```python
import random
import time
from typing import List
from models import Sample, TestCase

# Giả lập hàm tạo prompt từ samples, format, conditions (sử dụng "4o-mini")
def generate_prompt_from_samples(fmt: str, samples: List[Sample], conditions: str) -> str:
    base = f"Format: {fmt}\n"
    sample_str = "\n".join([f"Input: {s.input} => Output: {s.output}" for s in samples])
    cond_str = f"Conditions: {conditions}" if conditions else ""
    prompt = f"{base}{sample_str}\n{cond_str}"
    return prompt

# Giả lập hàm tạo test cases từ prompt (sử dụng "4o-mini")
def generate_test_cases(prompt: str) -> List[TestCase]:
    # Ví dụ: tạo 5 test cases với dữ liệu ngẫu nhiên
    test_cases = []
    for i in range(1, 6):
        expected = f"Expected output {i}"
        # Giả lập actual_output bằng cách thêm chút nhiễu ngẫu nhiên
        actual = expected if random.random() > 0.3 else f"Wrong output {i}"
        is_correct = (actual == expected)
        similarity = 1.0 if is_correct else random.uniform(0.5, 0.7)
        test_cases.append(
            TestCase(
                input=f"Test input {i}",
                expected_output=expected,
                actual_output=actual,
                is_correct=is_correct,
                similarity_score=similarity
            )
        )
    return test_cases

# Giả lập hàm đánh giá prompt (sử dụng model LLM mạnh hơn "o1-min")
def evaluate_prompt(prompt: str, test_cases: List[TestCase]) -> (float, float):
    # Thời gian phản hồi giả lập
    start = time.time()
    # Accuracy tính theo tỉ lệ các test case đúng, có thể tính trung bình similarity
    correct_cases = sum(1 for tc in test_cases if tc.is_correct)
    accuracy = correct_cases / len(test_cases)
    response_time = time.time() - start
    return accuracy, response_time

# Giả lập hàm cập nhật prompt dựa trên feedback và kết quả đánh giá
def update_prompt(current_prompt: str, test_cases: List[TestCase], accuracy: float) -> str:
    # Ở đây ta mô phỏng cập nhật bằng cách thêm thông tin "Updated" vào prompt
    updated = current_prompt + "\n[Updated Prompt based on evaluation feedback]"
    return updated
```

### File: `backend/main.py`
```python
from fastapi import FastAPI
from models import PromptRequest, PromptResponse, FeedbackRequest, OptimizationHistory
from utils import generate_prompt_from_samples, generate_test_cases, evaluate_prompt, update_prompt

app = FastAPI(title="Auto Prompting Tool API")

MAX_ITERATION = 5  # số vòng lặp tối đa

@app.post("/api/generate-prompt", response_model=PromptResponse)
def generate_prompt_endpoint(request: PromptRequest):
    # Bước 1: Tự viết Prompt (dùng 4o-mini)
    prompt = generate_prompt_from_samples(request.format, request.samples, request.conditions)
    iteration = request.iteration
    optimization_history = []

    # Bước 2: Tự tạo test cases (dùng 4o-mini)
    test_cases = generate_test_cases(prompt)

    # Bước 3: Tự đánh giá kết quả (dùng model LLM mạnh hơn)
    accuracy, response_time = evaluate_prompt(prompt, test_cases)
    optimization_history.append(OptimizationHistory(
        iteration=iteration,
        accuracy=accuracy,
        response_time=response_time
    ))

    # Lặp lại bước 1 và 2 (tự cập nhật prompt) nếu chưa đạt điều kiện (accuracy < 90%)
    while accuracy < 0.9 and iteration < MAX_ITERATION:
        iteration += 1
        # Cập nhật prompt dựa trên feedback nội bộ (điều chỉnh tự động)
        prompt = update_prompt(prompt, test_cases, accuracy)
        test_cases = generate_test_cases(prompt)
        accuracy, response_time = evaluate_prompt(prompt, test_cases)
        optimization_history.append(OptimizationHistory(
            iteration=iteration,
            accuracy=accuracy,
            response_time=response_time
        ))

    return PromptResponse(
        generated_prompt=prompt,
        test_cases=test_cases,
        accuracy=accuracy,
        response_time=response_time,
        iteration=iteration,
        optimization_history=optimization_history
    )

@app.post("/api/feedback")
def feedback_endpoint(request: FeedbackRequest):
    # Bước 4: Nhận feedback từ người dùng và lưu lại hoặc xử lý logic cập nhật
    # Ở đây chỉ demo log feedback. Thực tế có thể lưu vào cơ sở dữ liệu và kích hoạt quá trình update prompt.
    print("Feedback nhận được:", request.feedback)
    return {"message": "Feedback đã được nhận!"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)
```

### File: `backend/requirements.txt`
```
fastapi
uvicorn
pydantic
```

### File: `backend/Dockerfile`
```dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

EXPOSE 8000

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

---

## 2. Frontend (React)

### Cấu trúc thư mục mẫu:
```
frontend/
├── public/
│   └── index.html
├── src/
│   └── App.tsx
├── package.json
└── Dockerfile
```

### File: `frontend/public/index.html`
```html
<!DOCTYPE html>
<html lang="vi">
<head>
  <meta charset="UTF-8">
  <title>Auto Prompting Tool</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
</head>
<body>
  <div id="root"></div>
</body>
</html>
```

### File: `frontend/src/App.tsx`
Ví dụ dưới đây bao gồm các bước:  
- Gửi dữ liệu để tạo prompt và nhận kết quả (bao gồm prompt, test cases, accuracy, response time, iteration, optimization history).  
- Hiển thị kết quả và bảng test cases.  
- Nhận feedback từ người dùng qua một form riêng.
```tsx
import React, { useState, ChangeEvent, FormEvent } from 'react'
import axios from 'axios'

interface Sample {
  input: string
  output: string
}

interface TestCase {
  input: string
  expected_output: string
  actual_output: string
  is_correct: boolean
  similarity_score: number
}

interface OptimizationHistory {
  iteration: number
  accuracy: number
  response_time: number
}

interface PromptResponseData {
  generated_prompt: string
  test_cases: TestCase[]
  accuracy: number
  response_time: number
  iteration: number
  optimization_history: OptimizationHistory[]
}

function App() {
  const [format, setFormat] = useState('')
  const [samples, setSamples] = useState<Sample[]>([{ input: '', output: '' }])
  const [conditions, setConditions] = useState('')
  const [generatedPrompt, setGeneratedPrompt] = useState('')
  const [testCases, setTestCases] = useState<TestCase[]>([])
  const [accuracy, setAccuracy] = useState(0)
  const [responseTime, setResponseTime] = useState(0)
  const [iteration, setIteration] = useState(0)
  const [optimizationHistory, setOptimizationHistory] = useState<OptimizationHistory[]>([])
  const [feedback, setFeedback] = useState('')
  const [isLoading, setIsLoading] = useState(false)

  const API_URL = import.meta.env.VITE_API_URL || 'http://localhost:8000'

  const handleAddSample = () => {
    setSamples([...samples, { input: '', output: '' }])
  }

  const handleSampleChange = (index: number, field: 'input' | 'output', value: string) => {
    const newSamples = [...samples]
    newSamples[index][field] = value
    setSamples(newSamples)
  }

  const handleTextChange = (e: ChangeEvent<HTMLTextAreaElement | HTMLInputElement>) => {
    const { id, value } = e.target
    if (id === 'format') setFormat(value)
    if (id === 'conditions') setConditions(value)
    if (id === 'feedback') setFeedback(value)
  }

  const handleInputChange = (
    e: ChangeEvent<HTMLInputElement>,
    index: number,
    field: 'input' | 'output'
  ) => {
    handleSampleChange(index, field, e.target.value)
  }

  // Gửi dữ liệu để tạo prompt và nhận kết quả (các bước 1-3 và tự lặp)
  const handleGeneratePrompt = async (e: FormEvent) => {
    e.preventDefault()
    try {
      setIsLoading(true)
      const response = await axios.post<PromptResponseData>(`${API_URL}/api/generate-prompt`, {
        format,
        samples,
        conditions,
        iteration: 0
      })
      const data = response.data
      setGeneratedPrompt(data.generated_prompt)
      setTestCases(data.test_cases)
      setAccuracy(data.accuracy)
      setResponseTime(data.response_time)
      setIteration(data.iteration)
      setOptimizationHistory(data.optimization_history)
    } catch (error) {
      console.error('Error:', error)
    } finally {
      setIsLoading(false)
    }
  }

  // Gửi feedback từ người dùng (Bước 4)
  const handleFeedbackSubmit = async () => {
    try {
      await axios.post(`${API_URL}/api/feedback`, {
        prompt: generatedPrompt,
        feedback: feedback
      })
      alert('Feedback đã được gửi!')
    } catch (error) {
      console.error('Feedback error:', error)
    }
  }

  return (
    <div className="container mx-auto p-4">
      <h1 className="text-2xl font-bold mb-4">Auto Prompting Tool</h1>
      
      <form onSubmit={handleGeneratePrompt}>
        <div className="mb-4">
          <label htmlFor="format" className="block mb-2">Prompt Format:</label>
          <textarea
            id="format"
            className="w-full p-2 border rounded"
            value={format}
            onChange={handleTextChange}
          />
        </div>

        <div className="mb-4">
          <label className="block mb-2">Samples:</label>
          {samples.map((sample, index) => (
            <div key={index} className="mb-2 flex gap-2">
              <input
                type="text"
                className="flex-1 p-2 border rounded"
                placeholder="Input"
                value={sample.input}
                onChange={(e) => handleInputChange(e, index, 'input')}
              />
              <input
                type="text"
                className="flex-1 p-2 border rounded"
                placeholder="Output"
                value={sample.output}
                onChange={(e) => handleInputChange(e, index, 'output')}
              />
            </div>
          ))}
          <button type="button" className="bg-blue-500 text-white px-4 py-2 rounded" onClick={handleAddSample}>
            Add Sample
          </button>
        </div>

        <div className="mb-4">
          <label htmlFor="conditions" className="block mb-2">Additional Conditions:</label>
          <textarea
            id="conditions"
            className="w-full p-2 border rounded"
            value={conditions}
            onChange={handleTextChange}
          />
        </div>

        <button
          type="submit"
          className={`bg-green-500 text-white px-4 py-2 rounded ${isLoading ? 'opacity-50 cursor-not-allowed' : ''}`}
          disabled={isLoading}
        >
          {isLoading ? 'Generating...' : 'Generate Prompt'}
        </button>
      </form>

      {generatedPrompt && (
        <div className="mt-4">
          <h2 className="text-xl font-bold">Generated Prompt:</h2>
          <pre className="p-4 bg-gray-100 rounded whitespace-pre-wrap">{generatedPrompt}</pre>
          
          <div className="mt-2 grid grid-cols-3 gap-4">
            <div>
              <p className="font-bold">Accuracy:</p>
              <p>{(accuracy * 100).toFixed(2)}%</p>
            </div>
            <div>
              <p className="font-bold">Response Time:</p>
              <p>{responseTime.toFixed(2)}s</p>
            </div>
            <div>
              <p className="font-bold">Iterations:</p>
              <p>{iteration}</p>
            </div>
          </div>
        </div>
      )}

      {optimizationHistory.length > 0 && (
        <div className="mt-4">
          <h2 className="text-xl font-bold">Optimization History:</h2>
          <div className="grid gap-4">
            {optimizationHistory.map((history, index) => (
              <div key={index} className="p-4 bg-gray-100 rounded">
                <p><strong>Iteration {history.iteration}:</strong></p>
                <p>Accuracy: {(history.accuracy * 100).toFixed(2)}%</p>
                <p>Response Time: {history.response_time.toFixed(2)}s</p>
              </div>
            ))}
          </div>
        </div>
      )}

      {testCases.length > 0 && (
        <div className="mt-4">
          <h2 className="text-xl font-bold">Test Cases:</h2>
          <div className="grid gap-4">
            {testCases.map((test, index) => (
              <div key={index} className={`p-4 rounded ${test.is_correct ? 'bg-green-100' : 'bg-red-100'}`}>
                <p><strong>Input:</strong> {test.input}</p>
                <p><strong>Expected:</strong> {test.expected_output}</p>
                <p><strong>Actual:</strong> {test.actual_output}</p>
                <p><strong>Similarity:</strong> {(test.similarity_score * 100).toFixed(2)}%</p>
              </div>
            ))}
          </div>
        </div>
      )}

      {/* Phần nhận feedback từ người dùng (Bước 4) */}
      <div className="mt-6">
        <h2 className="text-xl font-bold mb-2">User Feedback</h2>
        <textarea
          id="feedback"
          className="w-full p-2 border rounded mb-2"
          placeholder="Nhập feedback của bạn"
          value={feedback}
          onChange={handleTextChange}
        />
        <button className="bg-purple-500 text-white px-4 py-2 rounded" onClick={handleFeedbackSubmit}>
          Submit Feedback
        </button>
      </div>
    </div>
  )
}

export default App
```

### File: `frontend/package.json`
Đảm bảo có các dependencies cần thiết (ví dụ React, TypeScript, axios, …)
```json
{
  "name": "auto-prompting-tool",
  "version": "1.0.0",
  "private": true,
  "scripts": {
    "start": "vite",
    "build": "vite build"
  },
  "dependencies": {
    "axios": "^1.3.0",
    "react": "^18.2.0",
    "react-dom": "^18.2.0"
  },
  "devDependencies": {
    "typescript": "^4.9.0",
    "vite": "^4.0.0",
    "@types/react": "^18.0.0",
    "@types/react-dom": "^18.0.0"
  }
}
```

### File: `frontend/Dockerfile`
```dockerfile
FROM node:18-alpine

WORKDIR /app

COPY package*.json ./
RUN npm install

COPY . .

EXPOSE 5173

CMD ["npm", "run", "start"]
```

---

## 3. Tổng Kết Quy Trình

1. **Tự viết Prompt (sử dụng model 4o-mini):**  
   - Người dùng nhập thông tin (format, samples, conditions) ở giao diện frontend.  
   - Gửi request đến endpoint `/api/generate-prompt` ở backend.
2. **Tự tạo Test Cases (dùng 4o-mini):**  
   - Backend sinh test cases dựa trên prompt vừa tạo.
3. **Tự đánh giá kết quả (dùng model LLM mạnh hơn):**  
   - Backend đánh giá prompt dựa trên các test case, tính toán accuracy và response time.  
   - Nếu chưa đạt yêu cầu (accuracy < 90%), backend tự cập nhật prompt (bước 1 và 2) theo một vòng lặp cho đến đạt điều kiện hoặc đến số vòng tối đa.
4. **User đánh giá và feedback lại:**  
   - Frontend hiển thị kết quả (prompt, test cases, evaluation metrics, optimization history).  
   - Người dùng nhập feedback và gửi qua endpoint `/api/feedback`.
5. **Update Prompt:**  
   - Backend có thể sử dụng feedback của người dùng (hoặc logic tự động) để cập nhật prompt.
6. **Lặp lại bước 3, 4, 5:**  
   - Quy trình đánh giá và cập nhật sẽ được lặp lại dựa trên yêu cầu hoặc trigger từ phía người dùng.

Với hướng dẫn và code mẫu trên, bạn có thể triển khai hệ thống tự động tạo prompt, đánh giá và tối ưu theo quy trình được mô tả. Bạn có thể mở rộng logic, tích hợp các model thực sự và lưu trữ dữ liệu tùy theo nhu cầu dự án.